In [ ]:
import ftplib
from ftplib import FTP
import os
import netCDF4 as nc
from tqdm import tqdm
import xarray as xr

Ftp_Server_host = 'ftp2.psl.noaa.gov'
ftp = FTP(Ftp_Server_host)
ftp.login()
Ftp_source_files_path = '/Datasets/ncep.reanalysis.dailyavgs/surface/' # '/Datasets/ncep.reanalysis.dailyavgs/pressure/'
ftp.cwd(Ftp_source_files_path)
files_list_main = ftp.nlst() # files_list = [x for x in ftp.nlst() if x.split('.')[0] == 'omega']
print(len(files_list_main))

for var_name in ['uwnd', 'vwnd', 'omega', 'rhum', 'pres']:
    %mkdir Data
    print(f"***{var_name}***")
    files_list = [x for x in files_list_main if var_name in x]
    for filename in tqdm(files_list):
        ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'Data', filename), 'wb').write)
    if var_name != 'pres':
        ds = xr.open_mfdataset(os.path.join(os.getcwd(), 'Data', f'{var_name}.sig995.*.nc'),combine = 'by_coords', concat_dim="time")
    else:
        ds = xr.open_mfdataset(os.path.join(os.getcwd(), 'Data', f'{var_name}.sfc.*.nc'),combine = 'by_coords', concat_dim="time")

    ds.to_netcdf(f'{var_name}_combined.nc')
    print("***Done***")
    %rm -r Data
ftp.quit()